# Libraries

In [11]:
import numpy as np
from omegaconf import OmegaConf
import models
import loss_func
import train
import loader

# Load Data

In [6]:
cfg = OmegaConf.load("config.yaml")
cmd_cfg = OmegaConf.from_cli()
cfg = OmegaConf.merge(cfg, cmd_cfg)
print(OmegaConf.to_yaml(cfg))

seed: 42
optimize:
  optimizer: Adam
  lr: 0.001
train:
  epochs: 100
  batch_size: 32
dataset:
  path: MNIST
  test_size: 0.2
  train_size: 0.8
  val_size: 0.2
  num_workers: 2
--ip: 127.0.0.1
--stdin: 9011
--control: 9009
--hb: 9008
--Session:
  signature_scheme: hmac-sha256
  key: b"c9f5e196-600c-4bbb-b288-3b91bfe6c092"
--shell: 9010
--transport: tcp
--iopub: 9012
--f: c:\Users\aleja\AppData\Roaming\jupyter\runtime\kernel-v2-10024ED1tv4bAchKE.json



In [8]:
cfg.dataset.test_size

0.2

In [12]:
timeseries = np.random.rand(10, 1000)

train_loader, val_loader, test_loader = loader.generateLoaders(timeseries, 'pearson')

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices